## Librerías

In [ ]:
import pandas as pd
import numpy as np

#visualizacion

import seaborn as sns
import matplotlib.pyplot as plt
import itertools
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#modelos
import lightgbm as lgb
from lightgbm.callback import early_stopping
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree,  _tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.metrics import f1_score
from sklearn.ensemble import  RandomForestClassifier
from sklearn.impute import SimpleImputer
from joblib import Parallel, delayed
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import fbeta_score, make_scorer
from imblearn.under_sampling import TomekLinks


#base de datos
import duckdb
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

#optimizacion
import time
import optuna
from optuna.study import MaxTrialsCallback
from optuna.trial import TrialState
from optuna.visualization import plot_param_importances, plot_contour,  plot_slice, plot_optimization_history
import plotly.express as px

#kaggle y otros
import os
import kaggle 
import glob
from openpyxl import load_workbook
from datetime import datetime



#Shap values
import shap

In [ ]:
pip install shap

In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [ ]:
import random

numeros_random = random.sample(range(1, 1000), 10)
semillas =[ 400009,  500009, 500011, 500021, 600009]


## Datasets

### VM

In [ ]:
base_path = '/home/n_alba_dscience/'
modelos_path = base_path + 'buckets/b1/modelos/'
db_path = base_path + 'buckets/b1/db/'
datasets_path = base_path + 'buckets/b1/datasets/'
exp_path = base_path + 'buckets/b1/exp/'
save_path = base_path + 'buckets/b1/'


# 1 - dataset sólo con clase ternaria
dataset_clase_ternaria = datasets_path + 'competencia_02_ct.parquet'

# 2 - dataset con clase ternaria creada y feature engineering de lags 
dataset_lags_clase_ternaria = datasets_path + 'competencia_02_lags_y_clase_ternaria.parquet'



# 3 - dataset con clase ternaria creada y feature engineering de lags y deltas
dataset_lags_deltas_y_clase_ternaria = datasets_path + 'competencia_02_lags_deltas_y_clase_ternaria.parquet'



### Local

In [ ]:
base_path_l = r'C:\Users\Admin\Documents\1_Notebook\1_Estudio\1_UBA_Maestria_DS\1_Especializacion\1_Segundo_Semestre\DMEyF'
optuna_path_l = base_path_l + r'\Optuna\rf_segmentacion1\optimization_tree.db'

entregas_l = base_path_l + r'\Entregas\competencia_02'


db_path_l = base_path_l + r'datasets\competencia_2'

datasets_path_l = base_path_l + r'\datasets\competencia_2'




# 1 dataset sólo con clase ternaria
dataset_clase_ternaria_l = datasets_path_l + r'\competencia_02_ct.parquet'

# 2 dataset con clase ternaria creada y feature engineering de lags 
dataset_lags_clase_ternaria_l = datasets_path_l + r'\competencia_02_lags_y_clase_ternaria.parquet'

# 3 dataset con clase ternaria creada y feature engineering de lags y deltas
dataset_lags_deltas_y_clase_ternaria_l = datasets_path_l + r'\competencia_02_lags_deltas_y_clase_ternaria.parquet'

dataset_10_meses_l = datasets_path_l + r'\dataset_10_meses.parquet'

In [ ]:
dicc_psi = {'dataset_clase_ternaria':['cmobile_app_trx',
 'mpayroll',
 'mprestamos_prendarios',
 'Master_fultimo_cierre',
 'Master_Finiciomora',
 'Visa_mlimitecompra',
 'Visa_madelantopesos',
 'Visa_fultimo_cierre',
 'Visa_Finiciomora',
 'mcajeros_propios_descuentos',
 'mpayroll2',
 'internet',
 'mcomisiones_mantenimiento',
 'Master_Fvencimiento',
 'mrentabilidad_annual',
 'mcomisiones_otras',
 'mcomisiones',
 'mtransferencias_recibidas',
 'mrentabilidad']
            ,
  'dataset_clase_ternaria_para_transformar':['cmobile_app_trx',
 'mpayroll',
 'mprestamos_prendarios',
 'Master_fultimo_cierre',
 'Master_Finiciomora',
 'Visa_mlimitecompra',
 'Visa_madelantopesos',
 'Visa_fultimo_cierre',
 'Visa_Finiciomora',
 'internet',
 'mcomisiones_mantenimiento',
 'Master_Fvencimiento',
 'mrentabilidad_annual',
 'mcomisiones_otras',
 'mcomisiones',
 'mtransferencias_recibidas',
 'mrentabilidad'],
  
  
  'dataset_clase_ternaria_para_sacar': ['mcajeros_propios_descuentos', 'mpayroll2'],
  
'dataset_lags_deltas_y_clase_ternaria':['Visa_Fvencimiento_delta_2',
 'Visa_Finiciomora_delta_1',
 'Visa_Finiciomora_delta_2',
 'Master_fechaalta_delta_2',
 'Master_fechaalta_delta_1',
 'Visa_Fvencimiento_delta_1',
 'Master_fultimo_cierre_delta_1',
 'Master_Fvencimiento_delta_2',
 'mpayroll',
 'mpayroll2',
 'ccajas_depositos_delta_1',
 'mprestamos_prendarios',
 'Master_Fvencimiento_delta_1',
 'Master_fultimo_cierre_lag_1',
 'mcajeros_propios_descuentos',
 'Visa_mlimitecompra',
 'Visa_fultimo_cierre',
 'Visa_madelantopesos',
 'Visa_Finiciomora',
 'Visa_fechaalta_delta_2',
 'Visa_fultimo_cierre_lag_2',
 'Visa_fechaalta_delta_1',
 'mprestamos_prendarios_lag_1',
 'mpayroll2_lag_1',
 'mpayroll_lag_2',
 'mpayroll_lag_1',
 'mprestamos_prendarios_lag_2',
 'mpayroll2_lag_2',
 'mcajeros_propios_descuentos_lag_2',
 'mcajeros_propios_descuentos_lag_1',
 'Visa_fultimo_cierre_delta_1',
 'Master_Finiciomora',
 'Master_fultimo_cierre',
 'Visa_madelantopesos_lag_1',
 'Visa_madelantopesos_lag_2',
 'Visa_Finiciomora_lag_2',
 'cmobile_app_trx',
 'Visa_fultimo_cierre_lag_1',
 'Master_fultimo_cierre_lag_2',
 'Master_fultimo_cierre_delta_2',
 'Visa_fultimo_cierre_delta_2',
 'internet',
 'mcomisiones_mantenimiento_lag_1',
 'mpayroll_delta_1',
 'ccomisiones_otras_delta_2',
 'mcomisiones_mantenimiento',
 'mrentabilidad_annual_lag_2',
 'Visa_Finiciomora_lag_1',
 'mcomisiones_mantenimiento_lag_2',
 'ccomisiones_otras_delta_1',
 'mrentabilidad_annual_lag_1',
 'mcomisiones_otras_lag_2',
 'mrentabilidad_lag_2',
 'Master_Fvencimiento_lag_1',
 'mcomisiones_otras_lag_1',
 'cforex_lag_2',
 'mforex_sell_lag_2',
 'Master_Fvencimiento_lag_2',
 'Master_Fvencimiento',
 'mrentabilidad_annual',
 'Visa_mconsumosdolares_lag_2',
 'mrentabilidad_annual_delta_2',
 'cforex_sell_lag_2',
 'mcomisiones_otras',
 'mcomisiones',
 'mforex_sell_lag_1',
 'mrentabilidad_lag_1',
 'cforex_lag_1',
 'Visa_mconsumosdolares_lag_1',
 'mrentabilidad_annual_delta_1',
 'cpayroll_trx_delta_1',
 'chomebanking_transacciones_lag_2',
 'mtransferencias_recibidas_delta_1',
 'mtransferencias_recibidas_lag_2',
 'mautoservicio_lag_2',
 'Visa_mfinanciacion_limite_delta_2',
 'cforex_sell_lag_1',
 'ctransferencias_recibidas_lag_2',
 'active_quarter_delta_2',
 'mtransferencias_recibidas',
 'cforex_delta_2',
 'Visa_mlimitecompra_lag_2',
 'ccomisiones_otras_lag_2',
 'Visa_mlimitecompra_delta_2',
 'mforex_sell_delta_2',
 'Visa_mfinanciacion_limite_delta_1',
 'mtransferencias_recibidas_delta_2',
 'cforex_sell_delta_2',
 'mtarjeta_visa_consumo_lag_2',
 'mrentabilidad',
 'mtransferencias_emitidas_lag_2',
 'Visa_mfinanciacion_limite_lag_2',
 'chomebanking_transacciones_lag_1',
 'mautoservicio_lag_1'],
'delta_lags_y_clase_ternaria_para_transformar':['Visa_Fvencimiento_delta_2',
 'Visa_Finiciomora_delta_1',
 'Master_fechaalta_delta_2',
 'Master_fechaalta_delta_1',
 'Visa_Fvencimiento_delta_1',
 'Master_fultimo_cierre_delta_1',
 'Master_Fvencimiento_delta_2',
 'mpayroll',
 'mprestamos_prendarios',
 'Master_Fvencimiento_delta_1',
 'Master_fultimo_cierre_lag_1',
 'Visa_mlimitecompra',
 'Visa_fultimo_cierre',
 'Visa_madelantopesos',
 'Visa_Finiciomora',
 'Visa_fechaalta_delta_2',
 'Visa_fultimo_cierre_lag_2',
 'Visa_fechaalta_delta_1',
 'mprestamos_prendarios_lag_1',
 'mpayroll_lag_2',
 'mpayroll_lag_1',
 'mprestamos_prendarios_lag_2',
 'Visa_fultimo_cierre_delta_1',
 'Master_Finiciomora',
 'Master_fultimo_cierre',
 'Visa_Finiciomora_lag_2',
 'cmobile_app_trx',
 'Visa_fultimo_cierre_lag_1',
 'Master_fultimo_cierre_lag_2',
 'Master_fultimo_cierre_delta_2',
 'Visa_fultimo_cierre_delta_2',
 'internet',
 'mcomisiones_mantenimiento_lag_1',
 'mpayroll_delta_1',
 'ccomisiones_otras_delta_2',
 'mcomisiones_mantenimiento',
 'mrentabilidad_annual_lag_2',
 'Visa_Finiciomora_lag_1',
 'mcomisiones_mantenimiento_lag_2',
 'ccomisiones_otras_delta_1',
 'mrentabilidad_annual_lag_1',
 'mcomisiones_otras_lag_2',
 'mrentabilidad_lag_2',
 'Master_Fvencimiento_lag_1',
 'mcomisiones_otras_lag_1',
 'mforex_sell_lag_2',
 'mrentabilidad_annual',
 'mrentabilidad_annual_delta_2',
 'cforex_sell_lag_2',
 'mcomisiones_otras',
 'mcomisiones',
 'mrentabilidad_lag_1',
 'mrentabilidad_annual_delta_1',
 'chomebanking_transacciones_lag_2',
 'mtransferencias_recibidas_lag_2',
 'mautoservicio_lag_2',
 'Visa_mfinanciacion_limite_delta_2',
 'active_quarter_delta_2',
 'mtransferencias_recibidas',
 'Visa_mlimitecompra_lag_2',
 'ccomisiones_otras_lag_2',
 'Visa_mfinanciacion_limite_delta_1',
 'mtarjeta_visa_consumo_lag_2',
 'mrentabilidad',
 'mtransferencias_emitidas_lag_2',
 'Visa_mfinanciacion_limite_lag_2',
 'chomebanking_transacciones_lag_1',
 'mautoservicio_lag_1'],

'delta_lags_y_clase_ternaria_para_sacar': ['Visa_Finiciomora_delta_2',
 'mpayroll2',
 'ccajas_depositos_delta_1',
 'mcajeros_propios_descuentos',
 'mpayroll2_lag_1',
 'mpayroll2_lag_2',
 'mcajeros_propios_descuentos_lag_2',
 'mcajeros_propios_descuentos_lag_1',
 'Visa_madelantopesos_lag_1',
 'Visa_madelantopesos_lag_2',
 'cforex_lag_2',
 'Master_Fvencimiento_lag_2',
 'Master_Fvencimiento',
 'Visa_mconsumosdolares_lag_2',
 'mforex_sell_lag_1',
 'cforex_lag_1',
 'Visa_mconsumosdolares_lag_1',
 'cpayroll_trx_delta_1',
 'mtransferencias_recibidas_delta_1',
 'cforex_sell_lag_1',
 'ctransferencias_recibidas_lag_2',
 'cforex_delta_2',
 'Visa_mlimitecompra_delta_2',
 'mforex_sell_delta_2',
 'mtransferencias_recibidas_delta_2',
 'cforex_sell_delta_2'], 

 'lista_light_gbm_feature_importance':['ccajas_depositos_lag_1',
 'Master_msaldopesos_delta_2',
 'Master_msaldopesos_delta_1',
 'Master_msaldototal_delta_2',
 'Master_msaldodolares_delta_2',
 'Master_mpagado_delta_1',
 'Master_mpagospesos_delta_1',
 'Master_mpagospesos_delta_2',
 'mprestamos_hipotecarios_lag_2',
 'Master_mpagado_delta_2',
 'Master_madelantodolares_delta_1',
 'Master_madelantodolares_delta_2',
 'Visa_madelantodolares_delta_1',
 'Visa_madelantodolares_delta_2',
 'Master_mconsumototal_delta_2',
 'Master_cconsumos_delta_1',
 'Visa_mconsumototal_delta_1',
 'Visa_mconsumototal_delta_2',
 'Master_mpagado_lag_1',
 'Master_madelantodolares_lag_2',
 'Master_madelantodolares_lag_1',
 'Master_madelantopesos_lag_2',
 'Master_msaldodolares',
 'catm_trx_other',
 'ccajas_extracciones',
 'ccajas_depositos',
 'Master_madelantopesos_lag_1',
 'Master_mconsumototal_lag_2',
 'Master_mconsumototal_lag_1',
 'Master_mpagosdolares_lag_2',
 'Master_cadelantosefectivo_lag_1',
 'Master_mpagado_lag_2',
 'Master_mpagospesos_lag_2',
 'Master_mpagospesos_lag_1',
 'Master_madelantopesos',
 'Master_mpagado',
 'Master_cadelantosefectivo',
 'Master_mconsumototal',
 'cprestamos_hipotecarios_lag_2',
 'Visa_madelantodolares_lag_1',
 'mpagodeservicios_lag_2',
 'cprestamos_hipotecarios_lag_1',
 'mpagomiscuentas_lag_1',
 'cpagomiscuentas_lag_1',
 'ccajeros_propios_descuentos_lag_1',
 'cpagomiscuentas_lag_2',
 'mtarjeta_master_consumo_lag_1',
 'Master_cadelantosefectivo_lag_2',
 'Master_mlimitecompra',
 'Master_mpagosdolares_lag_1',
 'ctarjeta_master_descuentos_lag_1',
 'ctarjeta_visa_descuentos_lag_2',
 'ctarjeta_visa_descuentos_lag_1',
 'ccajeros_propios_descuentos_lag_2',
 'ccomisiones_mantenimiento_lag_1',
 'ccajas_depositos_lag_2',
 'ccajas_extracciones_lag_1',
 'Master_msaldodolares_delta_1',
 'ccajas_extracciones_lag_2',
 'ccajas_otras_lag_1',
 'ccajas_otras_lag_2',
 'matm_lag_1',
 'catm_trx_other_lag_1',
 'catm_trx_other_lag_2',
 'matm_other_lag_1',
 'ccajas_transacciones_lag_2',
 'Master_msaldopesos_lag_1',
 'Master_msaldodolares_lag_1',
 'Master_mconsumospesos_lag_1',
 'Master_msaldodolares_lag_2',
 'Master_mconsumospesos_lag_2',
 'Master_mconsumosdolares_lag_1',
 'Master_mconsumosdolares_lag_2',
 'cplazo_fijo_lag_1',
 'mplazo_fijo_dolares_lag_1',
 'mplazo_fijo_pesos_lag_1',
 'matm_delta_2',
 'catm_trx_other_delta_1',
 'catm_trx_other_delta_2',
 'matm_other_delta_2',
 'Master_madelantopesos_delta_1',
 'Master_madelantopesos_delta_2',
 'ccheques_depositados_rechazados_lag_2',
 'ccheques_depositados_rechazados_lag_1',
 'mcheques_emitidos_lag_1',
 'ccheques_emitidos_lag_1',
 'mcheques_depositados_lag_1',
 'mcheques_depositados_lag_2',
 'ccheques_depositados_lag_2',
 'mforex_sell',
 'mtarjeta_master_descuentos',
 'cforex_sell',
 'cforex',
 'cforex_buy',
 'ctarjeta_master_descuentos',
 'ccajeros_propios_descuentos',
 'ctarjeta_visa_descuentos',
 'mtarjeta_visa_descuentos',
 'ccheques_depositados_rechazados',
 'mcheques_depositados',
 'ccheques_depositados',
 'Visa_cadelantosefectivo_lag_1',
 'Visa_mconsumototal_lag_2',
 'Visa_mconsumototal_lag_1',
 'Visa_madelantodolares_lag_2',
 'mcheques_depositados_rechazados',
 'mtarjeta_master_descuentos_lag_1',
 'mtarjeta_master_descuentos_lag_2',
 'ctarjeta_master_descuentos_lag_2',
 'cpayroll_trx_delta_2',
 'mprestamos_prendarios_delta_2',
 'mprestamos_hipotecarios_delta_2',
 'mprestamos_hipotecarios_delta_1',
 'cprestamos_hipotecarios_delta_2',
 'cpayroll2_trx_lag_1',
 'cpayroll2_trx_lag_2',
 'mpagodeservicios_delta_2',
 'mplazo_fijo_pesos_lag_2',
 'mpagodeservicios_delta_1',
 'cpayroll2_trx_delta_1',
 'ctarjeta_master_debitos_automaticos_delta_1',
 'ctarjeta_master_debitos_automaticos_lag_1',
 'mpagomiscuentas_delta_2',
 'mpagodeservicios_lag_1',
 'cpagomiscuentas_delta_2',
 'ccajeros_propios_descuentos_delta_2',
 'mttarjeta_master_debitos_automaticos_lag_2',
 'cpagodeservicios_lag_1',
 'mttarjeta_master_debitos_automaticos_lag_1',
 'ctarjeta_master_debitos_automaticos_lag_2',
 'mtarjeta_visa_descuentos_delta_1',
 'ctarjeta_visa_descuentos_delta_2',
 'ctarjeta_visa_descuentos_delta_1',
 'mcajeros_propios_descuentos_delta_2',
 'cforex_delta_1',
 'cpayroll2_trx_delta_2',
 'mpayroll2_delta_1',
 'mpayroll2_delta_2',
 'mttarjeta_master_debitos_automaticos_delta_2',
 'ctarjeta_visa_transacciones_delta_2',
 'cprestamos_hipotecarios_delta_1',
 'ccajas_transacciones_lag_1',
 'mcheques_emitidos_rechazados_lag_2',
 'ccallcenter_transacciones_lag_2',
 'mcheques_depositados_rechazados_lag_2',
 'mcheques_depositados_rechazados_lag_1',
 'cseguro_accidentes_personales',
 'cseguro_vivienda',
 'cseguro_auto',
 'mextraccion_autoservicio_delta_2',
 'cextraccion_autoservicio_delta_1',
 'cforex_buy_delta_2',
 'mforex_buy_delta_1',
 'mforex_buy_delta_2',
 'mtarjeta_master_descuentos_delta_2',
 'mtarjeta_master_descuentos_delta_1',
 'ctarjeta_master_descuentos_delta_2',
 'ccheques_emitidos_delta_2',
 'ccheques_depositados_delta_2',
 'mcheques_depositados_delta_2',
 'ccheques_depositados_rechazados_delta_1',
 'cpayroll2_trx',
 'ccajas_transacciones_delta_2',
 'ccheques_emitidos_rechazados_delta_2',
 'mcheques_depositados_rechazados_delta_1',
 'ccheques_emitidos_rechazados_delta_1',
 'ccheques_depositados_rechazados_delta_2',
 'ccajas_extracciones_delta_2',
 'Visa_madelantodolares',
 'Visa_cadelantosefectivo_delta_2',
 'Visa_mconsumototal',
 'Visa_cadelantosefectivo',
 'active_quarter_lag_1',
 'Master_mpagosdolares_delta_2',
 'Master_mconsumototal_delta_1',
 'mcheques_depositados_delta_1',
 'ctarjeta_visa_debitos_automaticos',
 'mttarjeta_visa_debitos_automaticos',
 'Visa_msaldodolares_delta_2',
 'Visa_msaldodolares_delta_1',
 'Visa_msaldopesos_delta_1',
 'cforex_buy_lag_1',
 'mtransferencias_recibidas_lag_1',
 'cextraccion_autoservicio_lag_1',
 'mforex_buy_lag_1',
 'cforex_buy_lag_2',
 'mextraccion_autoservicio_lag_2',
 'active_quarter_lag_2',
 'ctarjeta_visa_transacciones_lag_2',
 'ccajas_otras_delta_2',
 'matm_delta_1',
 'Master_cconsumos_delta_2',
 'cextraccion_autoservicio_lag_2',
 'ctransferencias_recibidas_lag_1',
 'cprestamos_hipotecarios',
 'ctarjeta_master_transacciones',
 'mplazo_fijo_pesos',
 'cinversion1',
 'Master_cadelantosefectivo_delta_2',
 'Visa_msaldototal_delta_1',
 'cinversion2',
 'Master_cadelantosefectivo_delta_1',
 'Master_mconsumosdolares_delta_1',
 'Visa_madelantopesos_delta_2',
 'ccuenta_corriente',
 'Master_mlimitecompra_delta_1',
 'Visa_mconsumospesos_delta_2',
 'Visa_mlimitecompra_delta_1',
 'Visa_madelantopesos_delta_1',
 'cliente_vip']
}

In [ ]:
def ganancia(model, X, y, prop):
    y_pred = model.predict(X)
    return f1_score(y, y_pred)

In [ ]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

In [ ]:
def psi(expected, actual, buckets=10):

    def psi_formula(expected_prop, actual_prop):
        result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
        return result

    expected_not_null = expected.dropna()
    actual_not_null = actual.dropna()

    bin_edges = pd.qcut(expected_not_null, q=buckets, duplicates='drop').unique()
    bin_edges2 = [edge.left for edge in bin_edges] + [edge.right for edge in bin_edges]
    breakpoints = sorted(list(set(bin_edges2)))

    expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
    actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

    expected_prop = expected_counts / len(expected_not_null)
    actual_prop = actual_counts / len(actual_not_null)

    psi_not_null = psi_formula(expected_prop, actual_prop).sum()

    psi_null = 0

    if expected.isnull().sum() > 0 and actual.isnull().sum() > 0 :
      expected_null_percentage = expected.isnull().mean()
      actual_null_percentage = actual.isnull().mean()
      psi_null = psi_formula(expected_null_percentage, actual_null_percentage)

    return psi_not_null + psi_null

In [ ]:
# Creador clase ternaria

# df_parallel = pd.read_csv(r"C:\Users\Admin\Documents\1_Notebook\1_Estudio\1_UBA_Maestria_DS\1_Especializacion\1_Segundo_Semestre\DMEyF\Git\dmeyf2024\datasets\competencia_01_crudo.csv")

# start_time = time.time()

# # Crear la columna clase_ternaria usando la columna 'foto_mes' basada en la presencia real

# df_ternaria= duckdb.query('''

#     WITH siguiente_mes AS (

#         SELECT

#             numero_de_cliente,

#             foto_mes,

#             -- Obtener los meses siguientes en los que el cliente está presente

#             LEAD(foto_mes, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_proximo1,

#             LEAD(foto_mes, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_proximo2

#         FROM df_parallel

#     )

#     SELECT

#         numero_de_cliente,

#         foto_mes,

#         CASE

#             -- Si el cliente está en los dos meses consecutivos siguientes

#             WHEN foto_mes_proximo1 = foto_mes + 1 AND foto_mes_proximo2 = foto_mes + 2 THEN 'CONTINUA'

#             -- Si el cliente no está en el siguiente mes

#             WHEN foto_mes_proximo1 IS NULL OR foto_mes_proximo1 <> foto_mes + 1 THEN 'BAJA+1'

#             -- Si el cliente está en el siguiente mes pero no en el segundo mes consecutivo

#             WHEN foto_mes_proximo1 = foto_mes + 1 AND (foto_mes_proximo2 IS NULL OR foto_mes_proximo2 <> foto_mes + 2) THEN 'BAJA+2'

#             ELSE NULL

#         END AS clase_ternaria

#     FROM siguiente_mes

#     ORDER BY numero_de_cliente, foto_mes

#     ''')

# df_ternaria = df_ternaria.to_df()
# df_ternaria = df_ternaria[['numero_de_cliente', 'foto_mes', 'clase_ternaria']]


# df_parallel = df_parallel.merge(df_ternaria, on=['numero_de_cliente', 'foto_mes'], how='left')

# df_train = df_parallel


In [ ]:

# con = duckdb.connect(database=':memory:', read_only=False)

# ruta_csv = r"C:\Users\Admin\Documents\1_Notebook\1_Estudio\1_UBA_Maestria_DS\1_Especializacion\1_Segundo_Semestre\DMEyF\datasets\competencia_2\competencia_02_f_e_lags.parquet"

# con.execute(f"CREATE TABLE df_train AS SELECT * FROM read_parquet('{ruta_csv}')")

In [ ]:
def calcular_medidas(X_train, Y_train, w_train, X_test, Y_test, w_test):

    lista_medidas = [accuracy_score, precision_score, recall_score, f1_score, roc_auc_score]
    train_data = lgb.Dataset(X_train, label=Y_train, weight=w_train)
    train_data = lgb.Dataset(X_test, label=X_test, weight=w_test)
    df_voting = pd.DataFrame()

    for semilla in semillas:
        params = {
            'objective': 'binary',  
            'metric': 'binary_logloss',  
            'seed': semilla, }
        model_default = lgb.train(params, train_data)   
        
        df_voting[f'semilla_{semilla}'] = model_default.predict(X_test)

    df_voting['promedio'] = df_voting.mean(axis=1)
    y_pred_default = df_voting['promedio']
    y_pred_labels = (y_pred_default >= 0.025).astype(int)
    dicc_medidas = {}
    
    for medida in lista_medidas:
     dicc_medidas[medida.__name__] = medida(Y_test, y_pred_labels)
    ganancia = lgb_gan_eval(y_pred_default, test_data)
    dicc_medidas['ganancia'] = ganancia[1]
    
    return dicc_medidas

In [ ]:
def anotar_excel(path_excel, dicc_medidas):
    if excel == True:
    
        wb = load_workbook(path_excel)
        ws = wb.active

        header_row = 1  
        column_mapping = {cell.value: cell.column for cell in ws[header_row]}
    
    row_to_update = None

    for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=1):
        cell = row[0]  # La primera celda de la fila
        if cell.value is None:  # Si el valor de la celda está vacía
            print(f"Primera celda vacía en la fila {cell.row}")
            row_to_update = cell.row
            break

    # Si no se encontró ninguna celda vacía, agregar al final
    if row_to_update is None:
        print("No se encontro celda vacía, agregando al final")
        row_to_update = ws.max_row + 1  # Agregar en una nueva fila al final

    print(f"Fila a actualizar: {row_to_update}")
    fila_actualizar = row_to_update  # Fila donde escribir los datos

    for col_name, value in dicc_medidas.items():
        if col_name in column_mapping:  # Asegurarse de que la columna existe
            col_idx = column_mapping[col_name]
            ws.cell(row=fila_actualizar, column=col_idx, value=value)

    # Guardar los cambios
    wb.save(path_excel)
    
    return print (f"se actualizó fila {fila_actualizar} en el excel")

In [ ]:
def calcular_medidas(X_train, Y_train, w_train, X_test, Y_test, w_test):

    lista_medidas = [accuracy_score, precision_score, recall_score, f1_score, roc_auc_score]
    train_data = lgb.Dataset(X_train, label=Y_train, weight=w_train)
    test_data = lgb.Dataset(X_test, label=Y_test, weight=w_test)
    df_voting = pd.DataFrame()

    for semilla in semillas:
        params = {
            'objective': 'binary',  
            'metric': 'binary_logloss',  
            'seed': semilla, }
        model_default = lgb.train(params, train_data)   
        
        df_voting[f'semilla_{semilla}'] = model_default.predict(X_test)

    df_voting['promedio'] = df_voting.mean(axis=1)
    y_pred_default = df_voting['promedio']
    y_pred_labels = (y_pred_default >= 0.025).astype(int)
    
    dicc_medidas = {}
    
    for medida in lista_medidas:
     dicc_medidas[medida.__name__] = medida(Y_test, y_pred_labels)
    ganancia = lgb_gan_eval(y_pred_default, test_data)
    dicc_medidas['ganancia'] = ganancia[1]
    
    return dicc_medidas

In [4]:
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


# Feature Engineering

In [ ]:
#Creadas las clases ternarias se procede a levantar dicho dataset en parquet

In [30]:
df_train = pd.read_parquet(dataset_clase_ternaria_l)

In [31]:
df_train.shape

(4735593, 155)

In [ ]:
# se hace lista con features a hacer feature engineering

### Status tarjeta

In [41]:
%%sql 


create or replace table df_train as
select
    *,
    (case when Visa_status = 0 then True end) as T_Visa_normal,
    (case when Master_status = 0 then True end) as T_Master_normal
from df_train;


create or replace table df_train as
        select *,
        CASE 
        WHEN Visa_status = 6 then 1
        WHEN Visa_status = 7 then 2 
        WHEN Visa_status = 8 then 3
        ELSE NULL
        END as Visa_status
        from df_train;

RuntimeError: If using snippets, you may pass the --with argument explicitly.
For more details please refer: https://jupysql.ploomber.io/en/latest/compose.html#with-argument


Original error message from DB driver:
(duckdb.duckdb.BinderException) Binder Error: Referenced column "Visa_status" not found in FROM clause!
Candidate bindings: "df_train.Visa_status_1"
LINE 4:     ...
                       ^
[SQL: create or replace table df_train as
select
    *,
    (case when Visa_status = 0 then True end) as T_Visa_normal,
    (case when Master_status = 0 then True end) as T_Master_normal
from df_train;]
(Background on this error at: https://sqlalche.me/e/20/f405)

If you need help solving this issue, send us a message: https://ploomber.io/community


In [ ]:
%%sql

alter table df_train drop column visa_status;
alter table df_train drop column Master_status;

,Success


### FE lags

In [43]:
lista_feature = ['active_quarter',
 'mrentabilidad',
 'mrentabilidad_annual',
 'mcuenta_corriente',
 'mcaja_ahorro',
 'mcuentas_saldo',
 'ctarjeta_debito_transacciones',
 'mautoservicio',
 'ctarjeta_visa_transacciones',
 'mtarjeta_visa_consumo',
 'mtarjeta_master_consumo',
 'cprestamos_personales',
 'mprestamos_prendarios',
 'cprestamos_hipotecarios',
 'mprestamos_hipotecarios',
 'cplazo_fijo',
 'mplazo_fijo_dolares',
 'mplazo_fijo_pesos',
 'cpayroll_trx',
 'mpayroll',
 'mpayroll2',
 'cpayroll2_trx',
 'ccuenta_debitos_automaticos',
 'mcuenta_debitos_automaticos',
 'ctarjeta_visa_debitos_automaticos',
 'mttarjeta_visa_debitos_automaticos',
 'ctarjeta_master_debitos_automaticos',
 'mttarjeta_master_debitos_automaticos',
 'cpagodeservicios',
 'mpagodeservicios',
 'mpagomiscuentas',
 'cpagomiscuentas',
 'ccajeros_propios_descuentos',
 'mcajeros_propios_descuentos',
 'ctarjeta_visa_descuentos',
 'mtarjeta_visa_descuentos',
 'ctarjeta_master_descuentos',
 'mtarjeta_master_descuentos',
 'ccomisiones_mantenimiento',
 'mcomisiones_mantenimiento',
 'ccomisiones_otras',
 'mcomisiones_otras',
 'cforex',
 'cforex_buy',
 'mforex_buy',
 'cforex_sell',
 'mforex_sell',
 'ctransferencias_recibidas',
 'mtransferencias_recibidas',
 'ctransferencias_emitidas',
 'mtransferencias_emitidas',
 'cextraccion_autoservicio',
 'mextraccion_autoservicio',
 'ccheques_depositados',
 'mcheques_depositados',
 'ccheques_emitidos',
 'mcheques_emitidos',
 'ccheques_depositados_rechazados',
 'mcheques_depositados_rechazados',
 'ccheques_emitidos_rechazados',
 'mcheques_emitidos_rechazados',
 'ccallcenter_transacciones',
 'chomebanking_transacciones',
 'ccajas_transacciones',
 'ccajas_consultas',
 'ccajas_depositos',
 'ccajas_extracciones',
 'ccajas_otras',
 'catm_trx',
 'matm',
 'catm_trx_other',
 'matm_other',
 'ctrx_quarter',
 'Master_Fvencimiento',
 'Master_msaldototal',
 'Master_msaldopesos',
 'Master_msaldodolares',
 'Master_mconsumospesos',
 'Master_mconsumosdolares',
 'Master_mlimitecompra',
 'Master_madelantopesos',
 'Master_madelantodolares',
 'Master_fultimo_cierre',
 'Master_mpagado',
 'Master_mpagospesos',
 'Master_mpagosdolares',
 'Master_fechaalta',
 'Master_mconsumototal',
 'Master_cconsumos',
 'Master_cadelantosefectivo',
 'Master_mpagominimo',
 'Visa_mfinanciacion_limite',
 'Visa_Fvencimiento',
 'Visa_Finiciomora',
 'Visa_msaldototal',
 'Visa_msaldopesos',
 'Visa_msaldodolares',
 'Visa_mconsumospesos',
 'Visa_mconsumosdolares',
 'Visa_mlimitecompra',
 'Visa_madelantopesos',
 'Visa_madelantodolares',
 'Visa_fultimo_cierre',
 'Visa_mpagado',
 'Visa_mpagospesos',
 'Visa_mpagosdolares',
 'Visa_fechaalta',
 'Visa_mconsumototal',
 'Visa_cconsumos',
 'Visa_cadelantosefectivo',
 'Visa_mpagominimo']

In [44]:
lag_columns = []
for variable in lista_feature:
    lag_columns.append(f"lag({variable}, 1) over (partition by numero_de_cliente order by foto_mes) as {variable}_lag_1")
    lag_columns.append(f"lag({variable}, 2) over (partition by numero_de_cliente order by foto_mes) as {variable}_lag_2")


lag_columns_sql = ",\n    ".join(lag_columns)


query = f"""
CREATE OR REPLACE TABLE df_train AS
SELECT *,
    {lag_columns_sql}
FROM df_train
"""

%sql {{query}}



,Success


In [ ]:
lag_columns = []
for variable in lista_feature:
    lag_columns.append(f"lag({variable}, 1) over (partition by numero_de_cliente order by foto_mes) as {variable}_lag_1")
    lag_columns.append(f"lag({variable}, 2) over (partition by numero_de_cliente order by foto_mes) as {variable}_lag_2")


lag_columns_sql = ",\n    ".join(lag_columns)


query = f"""
CREATE OR REPLACE TABLE df_train AS
SELECT *,
    {lag_columns_sql}
FROM df_train
"""

%sql {{query}}


In [58]:
path_dataset_lags_y_clase_ternaria = datasets_path_l+r'\competencia_02_lags_y_clase_ternaria.parquet'

In [59]:
path_dataset_lags_y_clase_ternaria

'C:\\Users\\Admin\\Documents\\1_Notebook\\1_Estudio\\1_UBA_Maestria_DS\\1_Especializacion\\1_Segundo_Semestre\\DMEyF\\datasets\\competencia_2\\competencia_02_lags_y_clase_ternaria.parquet'

In [77]:
delta_columns = []
for variable in lista_feature:
    nombre_variable = variable+'_delta_1'
    nombre_variable_2 = variable+'_delta_2'
    delta_columns.append(f"{variable} - {variable}_lag_1 as {nombre_variable}")
    delta_columns.append(f"{variable}_lag_1 - {variable}_lag_2 as {nombre_variable_2}")

delta_columns_sql = ",\n    ".join(delta_columns)


query = f'''CREATE OR REPLACE TABLE df_train AS
            select *,
            {delta_columns_sql}
            from df_train'''
%sql {{query}}
            

,Success


In [83]:
path_dataset_lags_deltas_y_clase_ternaria = datasets_path_l+r'\competencia_02_lags_deltas_y_clase_ternaria.parquet'
path_dataset_lags_deltas_y_clase_ternaria


'C:\\Users\\Admin\\Documents\\1_Notebook\\1_Estudio\\1_UBA_Maestria_DS\\1_Especializacion\\1_Segundo_Semestre\\DMEyF\\datasets\\competencia_2\\competencia_02_lags_deltas_y_clase_ternaria.parquet'

In [ ]:
# Guardo dataset con lags, deltas y clase ternaria

%%sql

COPY df_train TO '{{path_dataset_lags_deltas_y_clase_ternaria}}' (FORMAT 'parquet')


In [ ]:
# 2 - Cargar datos
# Opciones:
# a) dataset_clase_ternaria_l
# b) dataset_lags_clase_ternaria_l
# c) dataset_lags_deltas_y_clase_ternaria_l
# d) dataset_10_meses_l

df_train = pd.read_parquet(dataset_lags_deltas_y_clase_ternaria)



In [ ]:
df_train = df_train.drop(columns=dicc_psi['dataset_lags_deltas_y_clase_ternaria'])
df_train = df_train.drop(columns=dicc_psi['lista_light_gbm_feature_importance'])

## Formateo pre modelo

In [ ]:
if 'T_Visa_normal' in df_train.columns:
    df_train['T_Visa_normal'] = df_train['T_Visa_normal'].astype(bool)
if 'T_Master_normal'in df_train.columns:
    df_train['T_Master_normal'] = df_train['T_Master_normal'].astype(bool)


In [ ]:
# 3 - parametros para modelo
ganancia_acierto = 273000
costo_estimulo = 7000

mes_train = 202106
mes_test = 202108


In [ ]:
data = df_train

In [ ]:

data['clase_peso'] = 1.0
data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001
data['clase_binaria'] = np.where(data['clase_ternaria']=='CONTINUA', 0, 1)
data['foto_mes'].unique()
meses = [202011, 202012, 202101, 202102, 202103, 202104,202105,202106]


df_train = data[data['foto_mes']<=mes_train]
df_test = data[data['foto_mes']==mes_test]
clase_peso = df_train['clase_peso']
X_train = df_train.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_train =df_train['clase_binaria']
X_test = df_test.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_test =df_test['clase_binaria']
w_train = df_train['clase_peso']
w_test = df_test['clase_peso']
df_train['foto_mes'].unique()

array([202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106],
      dtype=int64)

## Predicciones

In [ ]:
#Mejores parámetros obtenidos con Optimización Bayesiana

best_params = {'num_leaves': 1796,
 'learning_rate': 0.049283676079631966,
 'min_data_in_leaf': 9,
 'feature_fraction': 0.44425115760554584,
 'bagging_fraction': 0.40692971865213867}

In [ ]:
numeros_random = np.random.randint(0, 100000, 20)

In [ ]:


train_data = lgb.Dataset(X_train,
                            label=Y_train,
                            weight=w_train)



best_iter = 1789  # Mejor iteración obtenida con Optimización Bayesiana
df_modelos = pd.DataFrame
params = {
'objective': 'binary',
'boosting_type': 'gbdt',
'first_metric_only': True,
'boost_from_average': True,
'feature_pre_filter': False,
'max_bin': 31,
'num_leaves': best_params['num_leaves'],
'learning_rate': best_params['learning_rate'],
'min_data_in_leaf': best_params['min_data_in_leaf'],
'feature_fraction': best_params['feature_fraction'],
'bagging_fraction': best_params['bagging_fraction'],
'seed': semillas[0],
'verbose': 0
}
        
        
model = lgb.train(params,
                train_data,
                num_boost_round=best_iter)

y_pred = model.predict(X_test)



[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [ ]:

X_test['Probabilidad'] = y_pred

tb_entrega = X_test.sort_values(by='Probabilidad', ascending=False)

tb_entrega['Predicted'] = 0

envios = 12300
tb_entrega.iloc[:envios, tb_entrega.columns.get_loc('Predicted')] = 1

resultados = tb_entrega[["numero_de_cliente", 'Predicted']].reset_index(drop=True)

print("Cantidad de clientes {}".format(envios))
num_subida_kaggle = 27
nombre_archivo = '\entrega_0{}.csv'.format(num_subida_kaggle)
entrega_final = f'{save_path}{nombre_archivo}'
resultados.to_csv(entrega_final, index=False)

Cantidad de clientes 14100
